In [1]:
import numpy as np
import cv2
import time
import datetime
import os
import shutil
import dill
import pandas as pd
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.models import Sequential, load_model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
from sklearn.neighbors.kde import KernelDensity
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

Using TensorFlow backend.


In [105]:
def clear_photos(dir_path):
    for l in os.walk(dir_path):
        i = 0
        if len(l[2]) > 0:
            for photo in tqdm_notebook(l[2], desc=l[0]):\
                os.remove(os.path.join(l[0], photo))

clear_photos('photos')

# 1. Cut video to photos

In [106]:
def video_to_photos(input_file, output_dir):
    cap = cv2.VideoCapture(input_file)
    if cap.isOpened() == False:
        raise Exception('Video stream doesn\'t open!')
    fps = np.round(cap.get(cv2.CAP_PROP_FPS), 0)
    sample_size = 0
    frame_num = 1
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            if frame_num % fps == 0:
                cv2.imwrite('{}/{}.jpg'.format(output_dir, frame_num), frame)
                sample_size += 1
        else:
            break
        frame_num += 1
        print('\rProgress: {}'.format(frame_num), end='')
    cap.release()
    print()
    return sample_size

In [107]:
samples = ('train', 'valid', 'test')
classes = ('0', '1')
photos_cnt = dict()
for s in samples:
    for c in classes:
        input_file = 'videos/input/{}/{}/video.mp4'.format(s, c)
        output_dir = 'photos/{}/{}'.format(s, c)
        photos_cnt['{}/{}'.format(s, c)] = video_to_photos(input_file, output_dir)
        print('Stage "{}, {}" has done'.format(s, c))

Progress: 60422
Stage "train, 0" has done
Progress: 32164
Stage "train, 1" has done
Progress: 49990
Stage "valid, 0" has done
Progress: 33042
Stage "valid, 1" has done
Progress: 48774
Stage "test, 0" has done
Progress: 31053
Stage "test, 1" has done


In [108]:
print(photos_cnt['train/1'] / (photos_cnt['train/0'] + photos_cnt['train/1']))
print(photos_cnt['valid/1'] / (photos_cnt['valid/0'] + photos_cnt['valid/1']))
print(photos_cnt['test/1'] / (photos_cnt['test/0'] + photos_cnt['test/1']))

0.3473752430330525
0.3979038670039754
0.3890977443609023


# 2. DataGen Creation

In [109]:
def preprocessing_image(image):
    image /= 255
    return image

In [110]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
#     brightness_range=[0.2, 1.8],
    zoom_range=[0.15, 1.15],
    horizontal_flip=True,
    preprocessing_function=preprocessing_image
)

In [111]:
target_size = (224, 224)

In [112]:
train_datagen = datagen.flow_from_directory(
    'photos/train',
    batch_size=16,
    target_size=target_size,
    class_mode='binary',
    seed=1992
)
valid_datagen = datagen.flow_from_directory(
    'photos/valid',
    batch_size=16,
    target_size=target_size,
    class_mode='binary',
    seed=1992
)

Found 3086 images belonging to 2 classes.
Found 2767 images belonging to 2 classes.


# 3. Transfer Learning

In [113]:
densenet = DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3)
)
# densenet.trainable = False

Instructions for updating:
Colocations handled automatically by placer.


In [114]:
model = Sequential()
model.add(densenet)
model.add(GlobalAveragePooling2D())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
_____

In [115]:
model.compile(Adam(), loss='binary_crossentropy')

In [116]:
checkpoint = ModelCheckpoint(filepath='model/checkpoint_best_model.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(patience=2, factor=0.5)
# tensorboard = TensorBoard(log_dir='logs/tensorboard', histogram_freq=2, update_freq='batch')
callbacks = [checkpoint, early_stopping, reduce_lr]

In [117]:
history = model.fit_generator(
    generator=train_datagen,
    steps_per_epoch=int(np.ceil(train_datagen.samples / train_datagen.batch_size)),
    verbose=2,
    epochs=1000,
    callbacks=callbacks,
    validation_data=valid_datagen,
    validation_steps=int(np.ceil(valid_datagen.samples / valid_datagen.batch_size)),
    workers=8,
#     use_multiprocessing=True,
#     class_weight={
#         0: 1.0,
#         1: 60422 / 32164
#     }
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
 - 81s - loss: 0.3584 - val_loss: 1.5288

Epoch 00001: val_loss improved from inf to 1.52875, saving model to model/checkpoint_best_model.h5
Epoch 2/1000
 - 54s - loss: 0.2441 - val_loss: 1.4233

Epoch 00002: val_loss improved from 1.52875 to 1.42328, saving model to model/checkpoint_best_model.h5
Epoch 3/1000
 - 54s - loss: 0.2277 - val_loss: 0.8037

Epoch 00003: val_loss improved from 1.42328 to 0.80367, saving model to model/checkpoint_best_model.h5
Epoch 4/1000
 - 53s - loss: 0.2334 - val_loss: 2.0686

Epoch 00004: val_loss did not improve from 0.80367
Epoch 5/1000
 - 53s - loss: 0.1908 - val_loss: 0.4568

Epoch 00005: val_loss improved from 0.80367 to 0.45683, saving model to model/checkpoint_best_model.h5
Epoch 6/1000
 - 53s - loss: 0.1783 - val_loss: 0.4438

Epoch 00006: val_loss improved from 0.45683 to 0.44381, saving model to model/checkpoint_best_model.h5
Epoch 7/1000
 - 53s - loss: 0.1538 - val_loss: 0.5558

Epoch

In [118]:
with open('model/history.pkl', 'wb') as f:
    dill.dump(history, f)

In [119]:
# with open('model/history.pkl', 'rb') as f:
#     history = dill.load(f)

# 4. Validation Curves 

In [120]:
train_loss = go.Scatter(
    x=list(range(1, len(history.history['loss']) + 1)),
    y=history.history['loss'],
    mode='lines+markers',
    name='Train loss',
    hoverinfo='y'
)
val_loss = go.Scatter(
    x=list(range(1, len(history.history['val_loss']) + 1)),
    y=history.history['val_loss'],
    mode='lines+markers',
    name='Validation loss',
    hoverinfo='y'
)

data = [train_loss, val_loss]
layout = go.Layout(
    title=dict(
        text='Learning curves'
    )
)
fig = go.Figure(data=data, layout=layout)

# fig.show()
pyo.plot(fig, filename='learning_curves.html')

'learning_curves.html'

In [121]:
model = load_model('model/checkpoint_best_model.h5')

In [122]:
# def get_test_batch(test_dir, target_img_size, batch_size=256):
#     for class_name in os.listdir(test_dir):
#         test_dir_class = os.path.join(test_dir, class_name)
#         image_names = []
#         true_labels = []
#         batch = []
#         images = os.listdir(test_dir_class)
#         for i in range(len(images)):
#             test_dir_class_image = os.path.join(test_dir_class, images[i])
#             img = load_img(test_dir_class_image)
#             img = img_to_array(img)
#             img = cv2.resize(img, target_img_size)
#             img = preprocessing_image(img)
#             image_names.append(images[i][:-4])
#             true_labels.append(class_name)
#             batch.append(img)
#             if (i + 1) % batch_size == 0 or i == len(images) - 1:
#                 yield {
#                     'image_names': image_names,
#                     'true_labels': true_labels,
#                     'batch': np.array(batch)
#                 }
#                 image_names = []
#                 true_labels = []
#                 batch = []

# test_dir = 'photos/test/'
# get_test_batch_gen = get_test_batch(test_dir, target_size)

# image_names = []
# y_true = []
# y_pred = []
# for batch in tqdm_notebook(get_test_batch_gen):
#     image_names.extend(batch['image_names'])
#     y_true.extend(list(map(int, batch['true_labels'])))
#     y_pred.extend(list(model.predict_on_batch(batch['batch']).ravel()))
# predictions = [image_names, y_true, y_pred]

# 5. ROC and PR curves

In [123]:
datagen_inference = ImageDataGenerator(
    preprocessing_function=preprocessing_image
)

In [124]:
def benchmark(func):
    import time
    def wrapper(*args, **kwargs):
        start_time = time.time()
        res = func(*args, **kwargs)
        end_time = time.time()
        delta = end_time - start_time
        minutes = int(delta / 60)
        seconds = int(delta - minutes * 60)
        print('Time of "{}": {} min. {} sec.'.format(func.__name__, minutes, seconds))
        return res
    return wrapper

In [125]:
@benchmark
def predict(model, datagen, directory, batch_size=256, target_size=(224, 224), workers=16):
    inference = datagen.flow_from_directory(
        directory,
        batch_size=batch_size,
        target_size=target_size,
        class_mode='binary',
        shuffle=False,
        seed=1992
    )
    image_names = list(map(lambda x: x[2:-4], inference.filenames))
    y_true = inference.classes
    y_pred = model.predict_generator(
        inference,
        steps=int(np.ceil(inference.samples / inference.batch_size)),
        workers=12,
        verbose=1
    ).ravel()
    return image_names, y_true, y_pred

In [126]:
image_names_train, y_train_true, y_train_pred = predict(model, datagen_inference, 'photos/train')

with open('predictions_train.pkl', 'wb') as f:
    dill.dump([image_names_train, y_train_true, y_train_pred], f)

image_names_valid, y_valid_true, y_valid_pred = predict(model, datagen_inference, 'photos/valid')

with open('predictions_valid.pkl', 'wb') as f:
    dill.dump([image_names_valid, y_valid_true, y_valid_pred], f)

image_names_test, y_test_true, y_test_pred = predict(model, datagen_inference, 'photos/test')

with open('predictions_test.pkl', 'wb') as f:
    dill.dump([image_names_test, y_test_true, y_test_pred], f)

Found 3086 images belonging to 2 classes.
13/13 [==============================] - ETA: 3: - ETA: 1: - ETA: 1: - ETA: 44s - ETA: 32 - ETA: 24 - ETA: 18 - ETA: 13 - ETA: 10 - ETA: 7 - ETA:  - ETA:  - 25s 2s/step
Time of "predict": 0 min. 25 sec.
Found 2767 images belonging to 2 classes.
11/11 [==============================] - ETA: 3: - ETA: 1: - ETA: 53s - ETA: 36 - ETA: 25 - ETA: 18 - ETA: 13 - ETA: 8 - ETA:  - ETA:  - 29s 3s/step
Time of "predict": 0 min. 29 sec.
Found 2660 images belonging to 2 classes.
11/11 [==============================] - ETA: 2: - ETA: 1: - ETA: 46s - ETA: 31 - ETA: 22 - ETA: 16 - ETA: 11 - ETA: 7 - ETA:  - ETA:  - 24s 2s/step
Time of "predict": 0 min. 24 sec.


In [127]:
def gini_score(roc_auc):
    return 2 * roc_auc - 1

In [128]:
roc_auc = roc_auc_score(y_train_true, y_train_pred)
gini = gini_score(roc_auc)
display('ROC AUC and GINI:', roc_auc, gini)

'ROC AUC and GINI:'

0.9961097874579437

0.9922195749158873

In [129]:
roc_auc = roc_auc_score(y_valid_true, y_valid_pred)
gini = gini_score(roc_auc)
display('ROC AUC and GINI:', roc_auc, gini)

'ROC AUC and GINI:'

0.9300213273320227

0.8600426546640454

In [130]:
roc_auc = roc_auc_score(y_test_true, y_test_pred)
gini = gini_score(roc_auc)
display('ROC AUC and GINI:', roc_auc, gini)

'ROC AUC and GINI:'

0.91812292827945

0.8362458565588999

In [131]:
def plot_curves(
    x_train,
    y_train,
    threshold_train,
    x_valid,
    y_valid,
    threshold_valid,
    x_test,
    y_test,
    threshold_test,
    curve_type='ROC',
    width=666,
    height=666
):
    
    trace_train = go.Scatter(
        x=x_train,
        y=y_train,
        mode='lines',
        name='Train',
        text=threshold_train
    )
    
    trace_valid = go.Scatter(
        x=x_valid,
        y=y_valid,
        mode='lines',
        name='Valid',
        text=threshold_valid
    )
    
    trace_test = go.Scatter(
        x=x_test,
        y=y_test,
        mode='lines',
        name='Test',
        text=threshold_test
    )

    data = [trace_train, trace_valid, trace_test]
    
    if curve_type == 'ROC':
        
        x_title = 'FPR'
        y_title = 'TPR'
        title = 'ROC Curves'
        
        trace_dot = go.Scatter(
            x=[0, 1],
            y=[0, 1],
            mode='lines',
            showlegend=False,
            line=dict(
                dash='dot'
            )
        )
        
        data.append(trace_dot)
        
    elif curve_type == 'PR':
        
        x_title = 'Recall'
        y_title = 'Precision'
        title = 'PR Curves'
    
    layout = go.Layout(
        title=dict(
            text=title
        ),
        hovermode='closest',
        width=width,
        height=height,
        xaxis=dict(
            title=dict(
                text=x_title
            )
        ),
        yaxis=dict(
            title=dict(
                text=y_title
            )
        )
    )
    fig = go.Figure(data=data, layout=layout)

#     fig.show()
    pyo.plot(fig, filename='{}_curves.html'.format(curve_type))

In [132]:
fpr_train, tpr_train, roc_thresholds_train = roc_curve(y_train_true, y_train_pred)
fpr_valid, tpr_valid, roc_thresholds_valid = roc_curve(y_valid_true, y_valid_pred)
fpr_test, tpr_test, roc_thresholds_test = roc_curve(y_test_true, y_test_pred)
plot_curves(
    fpr_train,
    tpr_train,
    roc_thresholds_train,
    fpr_valid,
    tpr_valid,
    roc_thresholds_valid,
    fpr_test,
    tpr_test,
    roc_thresholds_test
)

In [133]:
recall_train, precision_train, pr_thresholds_train = precision_recall_curve(y_train_true, y_train_pred)
recall_valid, precision_valid, pr_thresholds_valid = precision_recall_curve(y_valid_true, y_valid_pred)
recall_test, precision_test, pr_thresholds_test = precision_recall_curve(y_test_true, y_test_pred)
plot_curves(
    recall_train,
    precision_train,
    pr_thresholds_train,
    recall_valid,
    precision_valid,
    pr_thresholds_valid,
    recall_test,
    precision_test,
    pr_thresholds_test,
    curve_type='PR'
)

In [134]:
print('PR Train:', auc(recall_train, precision_train, reorder=True))
print('PR Valid:', auc(recall_valid, precision_valid, reorder=True))
print('PR Test:', auc(recall_test, precision_test, reorder=True))

PR Train: 0.26858177933332306
PR Valid: 0.4958372711727005
PR Test: 0.3165157198142953


# 6. Video Markup

In [135]:
def append_times_predictions_to_csv(times, predictions, filename):
    if len(times) != len(predictions):
        raise Exception('Arrays have different length! Times: {}, Predictions: {}'.format(len(times), len(predictions)))
    else:
        with open(filename, 'a') as f:
            for i in range(len(times)):
                f.write('{},{}\n'.format(times[i], predictions[i]))

In [136]:
%%time
input_file = 'videos/raw/valid/0/Real Madrid CF vs FC Barcelona  2-3  Full Match 23-04-17 HD.mp4'
output_file = 'time_and_predictions.csv'
batch_size = 512

cap = cv2.VideoCapture(input_file)
if cap.isOpened() == False:
    raise Exception('Video stream doesn\'t open!')
fps = cap.get(cv2.CAP_PROP_FPS)
frame_num = 0
times = []
batch = []
try:
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            times.append(frame_num / fps)
            frame = img_to_array(frame)
            frame = cv2.resize(frame, target_size)
            frame = preprocessing_image(frame)
            batch.append(frame)
            if (frame_num + 1) % batch_size == 0:
                batch = np.array(batch)
                predictions = list(model.predict_on_batch(batch).ravel())
                append_times_predictions_to_csv(times, predictions, output_file)
                times = []
                batch = []
            print('\rProgress: {}'.format(frame_num), end='')
            frame_num += 1
        else:
            batch = np.array(batch)
            predictions = list(model.predict_on_batch(batch).ravel())
            append_times_predictions_to_csv(times, predictions, output_file)
            print(' ... Done!')
            break
finally:
    cap.release()

Progress: 182014 ... Done!
Wall time: 44min 24s


In [137]:
df = pd.read_csv('time_and_predictions.csv', header=None)
df.head()

,0,1
0,0.000000,1.0
1,0.033367,1.0
2,0.066733,1.0
3,0.100100,1.0
4,0.133467,1.0


In [138]:
df.shape

(182015, 2)

In [139]:
df.columns = ['time', 'prediction']

In [140]:
df['second'] = df['time'].astype(int)
df.head()

,time,prediction,second
0,0.000000,1.0,0
1,0.033367,1.0,0
2,0.066733,1.0,0
3,0.100100,1.0,0
4,0.133467,1.0,0


In [141]:
df_seconds = df.groupby(['second'])[['prediction']].mean().reset_index()
df_seconds.head()

,second,prediction
0,0,0.976879
1,1,0.950476
2,2,0.999103
3,3,0.547093
4,4,0.098557


In [142]:
df_seconds['hms'] = df_seconds['second'].apply(lambda x: str(datetime.timedelta(seconds=x)))
df_seconds.head()

,second,prediction,hms
0,0,0.976879,0:00:00
1,1,0.950476,0:00:01
2,2,0.999103,0:00:02
3,3,0.547093,0:00:03
4,4,0.098557,0:00:04


In [143]:
trace = go.Scatter(
    x=df_seconds['hms'],
    y=df_seconds['prediction']
)
data = [trace]
layout = go.Layout(
    title=dict(
        text='Violation distribution'
    ),
    xaxis=dict(
        title=dict(
            text='Second'
        )
    ),
    yaxis=dict(
        title=dict(
            text='Average score per second'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='violation_distribution.html')

'violation_distribution.html'